In [1]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
from torch.utils.data import DataLoader
import kagglehub
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "sst2"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")
test_blank_data = load_from_disk(f"~/data/{DATASET}/test-blank-logits")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented", f"~/data/{DATASET}/test-blank-logits"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [6]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))
test_data_blank_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_blank_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [7]:
vocab = base.get_vocab(all_data_tokens)

In [ ]:
word_index = dict(zip(vocab, range(len(vocab))))

In [9]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [ ]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

16152


In [11]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 15775 words (377) misses


In [12]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))
test_data_blank_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_blank_tokens))

In [13]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))
test_blank_padded_data = list(map(lambda x: base.padd(x,60), test_data_blank_index))

In [14]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)
test_blank_data = test_blank_data.add_column("input_ids", test_blank_padded_data)

In [15]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed = False)

In [16]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)


In [17]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_embedd", logging_dir=f"~/logs/{DATASET}/bilstm-base_embedd", lr=.001,  epochs=10, batch_size=128)

In [18]:
base.reset_seed()

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.315200,0.557557,0.794725,0.816820,0.797118,0.791933
2,0.173500,0.423791,0.837156,0.837674,0.836764,0.836936
3,0.115400,0.439043,0.845183,0.845942,0.844731,0.844934
4,0.081200,0.564134,0.841743,0.843326,0.841100,0.841339
5,0.057500,0.681123,0.833716,0.836080,0.832923,0.833145
6,0.041600,0.790165,0.830275,0.835760,0.829081,0.829174


TrainOutput(global_step=2526, training_loss=0.13074229127909584, metrics={'train_runtime': 150.0154, 'train_samples_per_second': 3591.564, 'train_steps_per_second': 28.064, 'total_flos': 0.0, 'train_loss': 0.13074229127909584, 'epoch': 6.0})

In [ ]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)

In [22]:
trainer.evaluate(test_data)

{'eval_loss': 0.1866922378540039,
 'eval_accuracy': 0.9335560504825539,
 'eval_precision': 0.9319326072504803,
 'eval_recall': 0.9339344707430393,
 'eval_f1': 0.9328188831941853,
 'eval_runtime': 6.215,
 'eval_samples_per_second': 2167.352,
 'eval_steps_per_second': 17.056,
 'epoch': 6.0}

In [23]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base_embedd.pth")

In [24]:
test_blank_data.set_format(type="torch", columns=["input_ids"], device="cuda")
test_blank_dataloader = DataLoader(test_blank_data, batch_size=128, shuffle=False)
test_blank_logits = base.generate_logits(test_blank_dataloader, model, images=False)

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-base-embedd-test.tsv")

Created output file named: /home/jovyan/data/sst2/bilstm-base-embedd-test.tsv upload it to GLUE benchmark to obtain results!


Reálné skóre na pravé test části datasetu

![Real test score (GLUE Benchmark)](../imgs/sst2_BiLSTM_base_embedd_score.png)

**updated**

In [26]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_embedd", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill_embedd", lr=.001,  epochs=10, batch_size=128, lambda_param=.75, temp=5)

In [28]:
base.reset_seed()

In [29]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.724300,2.721900,0.802752,0.820301,0.804875,0.800723
2,0.707100,1.447206,0.857798,0.858030,0.857540,0.857672
3,0.406000,1.425392,0.853211,0.854082,0.852741,0.852960
4,0.256900,1.658160,0.846330,0.850218,0.845352,0.845599
5,0.181900,1.620506,0.854358,0.857523,0.853488,0.853776


TrainOutput(global_step=2105, training_loss=0.6552434527109467, metrics={'train_runtime': 142.2435, 'train_samples_per_second': 3787.801, 'train_steps_per_second': 29.597, 'total_flos': 0.0, 'train_loss': 0.6552434527109467, 'epoch': 5.0})

In [31]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)

In [32]:
trainer.evaluate(test_data)

{'eval_loss': 0.6829284429550171,
 'eval_accuracy': 0.9382331106161841,
 'eval_precision': 0.9368518420515255,
 'eval_recall': 0.9382484908850692,
 'eval_f1': 0.9374988358105241,
 'eval_runtime': 8.5111,
 'eval_samples_per_second': 1582.636,
 'eval_steps_per_second': 12.454,
 'epoch': 5.0}

In [33]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill_embedd.pth")

In [34]:
test_blank_logits = base.generate_logits(test_blank_dataloader, student_model, images=False)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-distill-embedd-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-distill-embedd-test.tsv upload it to GLUE benchmark to obtain results!


Reálné skóre na pravé test části datasetu

![Real test score (GLUE Benchmark)](../imgs/sst2_BiLSTM_distill_embedd_score.png)


**UPDATED**

In [53]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

In [54]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-aug_embedd", logging_dir=f"~/logs/{DATASET}/bilstm-base-aug_embedd", lr=.001,  epochs=10, batch_size=128)

In [55]:
base.reset_seed()

In [56]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)

In [57]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.129500,0.613966,0.836009,0.836590,0.835596,0.835774
2,0.044800,0.883028,0.821101,0.821212,0.821293,0.821097
3,0.025200,1.160420,0.827982,0.828483,0.828345,0.827978


: 

: 

In [ ]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.23352107405662537,
 'eval_accuracy': 0.9421677802524128,
 'eval_precision': 0.9412859245967904,
 'eval_recall': 0.9415141433774036,
 'eval_f1': 0.9413988215607065,
 'eval_runtime': 4.6863,
 'eval_samples_per_second': 2874.328,
 'eval_steps_per_second': 22.619,
 'epoch': 4.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-aug_embedd.pth")

In [ ]:
test_blank_logits = base.generate_logits(test_blank_dataloader, model, images=False)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-base-aug-embedd-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-base-aug-embedd-test.tsv upload it to GLUE benchmark to obtain results!


Reálné skóre na pravé test části datasetu

![Real test score (GLUE Benchmark)](../imgs/sst2_BiLSTM_base_aug_embedd_score.png)

In [ ]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-aug_embedd", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-aug_embedd", lr=.001,  epochs=10, batch_size=128, lambda_param=.75, temp=5)

In [ ]:
base.reset_seed()

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.476300,1.189721,0.883028,0.883325,0.883325,0.883028
2,0.150100,1.244355,0.866972,0.868168,0.867517,0.866947
3,0.099400,1.215065,0.856651,0.856929,0.856372,0.856514
4,0.074700,1.277731,0.849771,0.849724,0.849825,0.849747


TrainOutput(global_step=16676, training_loss=0.2001258811667029, metrics={'train_runtime': 608.4494, 'train_samples_per_second': 8770.326, 'train_steps_per_second': 68.518, 'total_flos': 0.0, 'train_loss': 0.2001258811667029, 'epoch': 4.0})

In [ ]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(16154, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=2, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.4353012442588806,
 'eval_accuracy': 0.9537490720118782,
 'eval_precision': 0.952483431863369,
 'eval_recall': 0.9540531728282269,
 'eval_f1': 0.9532060855391491,
 'eval_runtime': 5.9798,
 'eval_samples_per_second': 2252.596,
 'eval_steps_per_second': 17.726,
 'epoch': 4.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-aug_embedd.pth")

In [ ]:
test_blank_logits = base.generate_logits(test_blank_dataloader, student_model, images=False)
base.generate_real_test_file_sst2(test_blank_logits, f"{os.path.expanduser('~')}/data/{DATASET}/bilstm-distill-aug-embedd-test.tsv")

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

Created output file named: /home/jovyan/data/sst2/bilstm-distill-aug-embedd-test.tsv upload it to GLUE benchmark to obtain results!


Reálné skóre na pravé test části datasetu

![Real test score (GLUE Benchmark)](../imgs/ss2_BiLSTM_distil_aug_embedd_score.png)

Skóre učitelského modelu na reálné test části datasetu

![SST2 test score for best BERT model](../imgs/sst2_BERT_test_score.png)